***GENERATED CODE FOR shownoshowclassgb PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('AppointmentID')
        transformationDF = transformationDF.drop('PatientID')
        transformationDF = transformationDF.drop('PatientNumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'm', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentDate', 'transformation_label': 'Extract Date'}], 'feature': 'AppointmentDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'AppointmentDate'}, {'feature_label': 'AppointmentDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('AppointmentDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentType', 'transformation_label': 'String Indexer'}], 'feature': 'AppointmentType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': '*ESTABLISHED PATIENT', 'max': 'VIRTUAL', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AppointmentType'}, {'feature_label': 'AppointmentType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AppointmentType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeName', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': 'MS PG1', 'max': 'MS PG1', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeName'}, {'feature_label': 'PracticeName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeSpeciality', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeSpeciality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': 'Multi-Specialty', 'max': 'Multi-Specialty', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeSpeciality'}, {'feature_label': 'PracticeSpeciality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeSpeciality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeRegion', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeRegion', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': 'SouthEast', 'max': 'SouthEast', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeRegion'}, {'feature_label': 'PracticeRegion', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeRegion')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': 'SocCLEXX', 'max': 'SocWesXX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCategory'}, {'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCity', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': 'HOUSTON', 'max': 'Sugar Land', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCity'}, {'feature_label': 'FacilityCity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityState', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityState', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': 'TX', 'max': 'TX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityState'}, {'feature_label': 'FacilityState', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityState')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityZip', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityZip', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '771802086.76', 'stddev': '1822069.11', 'min': '770025351', 'max': '775053948', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityZip'}, {'feature_label': 'FacilityZip', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityZip')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': 'NoelAhmyyy, HassaXXX', 'max': 'NoelYouyyy, MeagaXXX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderName'}, {'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Speciality', 'transformation_label': 'String Indexer'}], 'feature': 'Speciality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': 'Allergy/Immun', 'max': 'Speech Pathologists', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Speciality'}, {'feature_label': 'Speciality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Speciality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderTaxonomyName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderTaxonomyName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': 'Allopathic & Osteopathic Physicians : Allergy & Immunology', 'max': 'Unspecified', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderTaxonomyName'}, {'feature_label': 'ProviderTaxonomyName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderTaxonomyName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '29246', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Work Comp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerCategory'}, {'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerCategory')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


def KNeighborsClassifier(df, labels, features, TrainingPercent=100):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    clf = KNeighborsClassifier()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    clf.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % clf.score(X_test, y_test))

    data = {'model': clf,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run shownoshowclassgbHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'IntelligentScheduling', 'optionalDB': 'IntelligentScheduling', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT sub2.* , \r\n\r\n       sub1.NoShowCount, \r\n       sub1.KeptCount, \r\n       sub1.[CanceledCount<=24hrs], \r\n       sub1.[CanceledCount>24hrs], \r\n       sub1.[Reschedule<=24hrs], \r\n       sub1.[Reschedule>24hrs] \r\nFROM \r\n\r\n( \r\n\r\nSELECT Distinct \r\n\r\n    A.AppointmentID, \r\n\r\n    A.PatientID, \r\n\r\n    PatientNumber, \r\n\r\n    Gender, \r\n\r\n    FLOOR(DATEDIFF(DAY, P.DateOfBirth, GETDATE()) / 365) AS PatientAge, \r\n\r\n    A.AppointmentDate, \r\n\r\n    DATEDIFF(day, A.CreatedDate,A.AppointmentDate) as days_AppDate_AppCreatDate, \r\n\r\n    A.AppointmentDuration, \r\n\r\n    AT.AppointmentTypeCategory as AppointmentType, \r\n\r\n    A.EligibilityFlag, \r\n\r\n    A.ConfirmationFlag, \r\n\r\n    PRC.PracticeName, \r\n\r\n    PRC.Speciality as PracticeSpeciality, \r\n\r\n    PRC.Region as PracticeRegion, \r\n\r\n    F.FacilityCategory, \r\n\r\n    F.City as FacilityCity, \r\n\r\n    F.State as FacilityState, \r\n\r\n    F.Zip as FacilityZip, \r\n\r\n    PR.ProviderCategory as ProviderName, \r\n\r\n    SP.SpecialityCategory as Speciality, \r\n\r\n    PTx.ProviderTaxonomyCategory as ProviderTaxonomyName, \r\n\r\n            CASE WHEN py.PayerCategory IS NOT NULL THEN py.PayerCategory ELSE 'NOT AVAILABLE' END as PayerCategory , \r\n\r\nCASE  \r\n\r\nWHEN  \r\n\r\n(AppointmentStatusCategory\u202f=\u202f'Cancel'\u202f\u202f  \r\n\r\nOR\u202fDeleteFlag\u202f=\u202f'Y')  \r\n\r\nAnd RescheduleFlag = 'N' \r\n\r\nTHEN\u202f0 \r\n\r\n    WHEN  \r\n\r\n        (AppointmentStatusCategory = 'Expected'   \r\n\r\n        AND RescheduleFlag = 'N'  \r\n\r\n        AND DeleteFlag = 'N')  \r\n\r\n    THEN 0 \r\n\r\n    ELSE 1 \r\n\r\nEND AS Label_NoKeepCancel\r\n \r\n\r\nFROM APPOINTMENT A \r\n\r\n    LEFT OUTER JOIN APPOINTMENTSTATUS  S on S.AppointmentStatusID=A.AppointmentStatusID \r\n\r\n    LEFT OUTER JOIN APPOINTMENTTYPE AT ON AT.AppointmentTypeID=A.AppointmentTypeID \r\n\r\n    LEFT OUTER JOIN ENCOUNTER EN ON EN.EncounterID=A.EncounterID \r\n\r\n    LEFT OUTER JOIN ENCOUNTERSTATUS ES ON ES.EncounterStatusID=EN.EncounterStatusID \r\n\r\n    Left outer JOIN PATIENT P ON P.PatientID=A.PatientID \r\n\r\n    LEFT OUTER JOIN PROVIDER PR ON PR.ProviderID=A.ProviderID \r\n\r\n    LEFT OUTER JOIN ProviderTaxonomy PTx ON PR.TaxonomyID=PTx.ProviderTaxonomyID \r\n\r\n    LEFT OUTER JOIN ReferringProvider RP ON RP.ReferringProviderID=A.ReferringProviderID \r\n\r\n    LEFT OUTER JOIN SPECIALITY SP ON SP.SpecialityID=PR.SpecialityID \r\n\r\n    LEFT OUTER JOIN FACILITY F ON F.Facilityid=A.FacilityID \r\n\r\n    LEFT OUTER JOIN PRACTICE PRC ON PRC.PracticeID=A.PracticeID \r\n\r\n    LEFT OUTER JOIN ChargeDetail  CD on CD.PatientID=A.PatientID \r\n\r\n    LEFT OUTER JOIN PAYER PY ON PY.PayerID=CD.PayerID \r\n\r\n) as sub2 \r\n\r\nleft join \r\n\r\n( \r\n\r\n   SELECT COALESCE(K.PatientID,NS.PatientID,C1.PatientID,c2.PatientID,R1.PatientID,R2.PatientID) AS PatientID, \r\n\r\n     ISNULL((NoShowCount),0) AS NoShowCount, \r\n\r\n     ISNULL((KeptCount),0) AS KeptCount, \r\n\r\n     ISNULL([CanceledCount<=24hrs],0) AS [CanceledCount<=24hrs], \r\n\r\n     ISNULL([CanceledCount>24hrs],0) AS [CanceledCount>24hrs], \r\n\r\n     ISNULL([Reschedule<=24hrs],0) AS [Reschedule<=24hrs], \r\n\r\n     ISNULL([Reschedule>24hrs],0) AS [Reschedule>24hrs] \r\n\r\n \r\n \r\n\r\n  FROM  \r\n\r\n( \r\n\r\n    SELECT  PatientID,COUNT(*) AS KeptCount \r\n\r\n    FROM APPOINTMENT A \r\n\r\n    LEFT OUTER JOIN APPOINTMENTSTATUS  S on S.AppointmentStatusID=A.AppointmentStatusID \r\n\r\n    WHERE   AppointmentStatusCategory = 'Kept' AND  AppointmentStatusCategory  <> 'Cancel'  \r\n\r\n    AND RescheduleFlag = 'N' AND DeleteFlag = 'N'  \r\n\r\n    GROUP BY PatientID \r\n\r\n)K  \r\n\r\nFULL OUTER JOIN \r\n\r\n( \r\n\r\n    SELECT  PatientID,COUNT(*) AS NoShowCount \r\n\r\n    FROM APPOINTMENT A \r\n\r\n    LEFT OUTER JOIN APPOINTMENTSTATUS  S on S.AppointmentStatusID=A.AppointmentStatusID \r\n\r\n    WHERE  AppointmentStatusCategory = 'Expected'  AND RescheduleFlag = 'N'   \r\n\r\n    AND DeleteFlag = 'N' AND  A.AppointmentDate < Getdate()   \r\n\r\n    GROUP BY PatientID \r\n\r\n )NS ON  K.PatientID = NS.PatientID \r\n\r\n  \r\n\r\nFULL OUTER JOIN  \r\n\r\n( \r\n\r\n    SELECT  PatientID, COUNT(*) AS [CanceledCount<=24hrs]  \r\n\r\n    FROM APPOINTMENT A \r\n\r\n    LEFT OUTER JOIN APPOINTMENTSTATUS  S on S.AppointmentStatusID=A.AppointmentStatusID \r\n\r\n     Where \r\n\r\n       (AppointmentStatusCategory = 'Cancel'   \r\n\r\n        OR DeleteFlag = 'Y')   \r\n\r\n        And RescheduleFlag = 'N' \r\n\r\n    AND  datediff(d,a.ModifiedDate,appointmentdate) <=1 \r\n\r\n    GROUP BY PatientID \r\n\r\n) C1 ON K.PatientID = C1.PatientID \r\n\r\n\r\nFULL OUTER JOIN \r\n\r\n( \r\n\r\n    SELECT Distinct PatientID,COUNT (*)  AS [CanceledCount>24hrs]  \r\n\r\n    FROM APPOINTMENT A \r\n\r\n    LEFT OUTER JOIN APPOINTMENTSTATUS  S on S.AppointmentStatusID=A.AppointmentStatusID \r\n\r\n     Where \r\n\r\n       (AppointmentStatusCategory = 'Cancel'   \r\n\r\n        OR DeleteFlag = 'Y')   \r\n\r\n        AND RescheduleFlag = 'N' \r\n\r\n        AND  datediff(d,a.ModifiedDate,appointmentdate) >1 \r\n\r\n        GROUP BY PatientID \r\n\r\n)C2  ON K.PatientID = C2.PatientID \r\n\r\nFULL OUTER JOIN \r\n\r\n( \r\n\r\n        SELECT PatientID, COUNT(*) AS [Reschedule<=24hrs]  from \r\n\r\n        ( \r\n\r\n                SELECT   \r\n\r\n                A.AppointmentID, \r\n\r\n                A.InitialAppointmentID, \r\n\r\n                PatientID, \r\n\r\n                AppointmentDate \r\n\r\n                FROM APPOINTMENT A \r\n\r\n                WHERE   A.AppointmentNumber IN  \r\n\r\n                    ( \r\n\r\n                    select   MAX(AppointmentNumber)  from Appointment T2  WITH (NOLOCK) \r\n\r\n                    where    RescheduleFlag = 'Y'AND  A.InitialAppointmentID = T2.InitialAppointmentID \r\n\r\n                    ) \r\n\r\n        ) DT          \r\n\r\n         INNER JOIN  \r\n\r\n                (        \r\n\r\n                SELECT InitialAppointmentID,AppointmentID,AppointmentDate,CreatedDate \r\n\r\n                FROM  Appointment WITH (NOLOCK) WHERE  RescheduleFlag = 'N' \r\n\r\n                )B    \r\n\r\n                   ON DT.InitialAppointmentID = B.InitialAppointmentID \r\n\r\n  \r\n\r\n     WHERE Datediff(day,B.CreatedDate,DT.AppointmentDate ) >= 0 and  Datediff(day,B.CreatedDate,DT.AppointmentDate ) <= 1 \r\n\r\n     GROUP BY PatientID \r\n\r\n) R1 ON K.PatientID = R1.PatientID \r\n\r\n\r\nFULL OUTER JOIN \r\n\r\n( \r\n\r\n        SELECT PatientID, COUNT(*) AS [Reschedule>24hrs]  from \r\n\r\n        ( \r\n\r\n                SELECT   \r\n\r\n                A.AppointmentID, \r\n\r\n                A.InitialAppointmentID, \r\n\r\n                PatientID, \r\n\r\n                AppointmentDate \r\n\r\n                FROM APPOINTMENT A \r\n\r\n                WHERE   A.AppointmentNumber IN  \r\n\r\n                    ( \r\n\r\n                    select   MAX(AppointmentNumber)  from Appointment T2  WITH (NOLOCK) \r\n\r\n                    where    RescheduleFlag = 'Y'AND  A.InitialAppointmentID = T2.InitialAppointmentID \r\n\r\n                    ) \r\n\r\n        ) DT          \r\n\r\n         INNER JOIN  \r\n\r\n                (        \r\n\r\n                SELECT InitialAppointmentID,AppointmentID,AppointmentDate,CreatedDate \r\n\r\n                FROM  Appointment WITH (NOLOCK) WHERE  RescheduleFlag = 'N' \r\n\r\n                )B    \r\n\r\n                   ON DT.InitialAppointmentID = B.InitialAppointmentID \r\n\r\n  \r\n\r\n     WHERE Datediff(day,B.CreatedDate,DT.AppointmentDate ) > 1 \r\n\r\n     GROUP BY PatientID \r\n\r\n) R2 ON K.PatientID = R2.PatientID \r\n\r\n \r\n \r\n\r\n) as sub1 \r\n\r\n\r\non  \r\n\r\nsub2.patientid= sub1.patientid \r\n\r\nWHERE sub2.AppointmentDate between  '1/01/2020' and '12/31/2020'", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run shownoshowclassgbHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "AppointmentID", "transformation_label": "String Indexer"}], "feature": "AppointmentID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "0004A337-E079-4B6F-AA8F-D3D8DE338EB1", "max": "FFFFB624-F7EA-4ABB-BFB8-916C8017D1E7", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AppointmentID"}, {"transformationsData": [{"feature_label": "PatientID", "transformation_label": "String Indexer"}], "feature": "PatientID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "000751C7-4B77-4979-A7CA-23DF28EF30C9", "max": "FFFD6E36-9784-4200-AC22-FAB5CC857C83", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientID"}, {"transformationsData": [{"feature_label": "PatientNumber", "transformation_label": "String Indexer"}], "feature": "PatientNumber", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "29246", "mean": "394375.15", "stddev": "198955.49", "min": "        1034", "max": "      603314", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientNumber"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "F", "max": "m", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientAge", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "53.67", "stddev": "21.19", "min": "0", "max": "114", "missing": "0"}, "updatedLabel": "PatientAge"}, {"transformationsData": [{"feature_label": "AppointmentDate", "transformation_label": "Extract Date"}], "feature": "AppointmentDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "AppointmentDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_AppDate_AppCreatDate", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "15.58", "stddev": "36.64", "min": "-112", "max": "371", "missing": "0"}, "updatedLabel": "days_AppDate_AppCreatDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AppointmentDuration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "17.56", "stddev": "27.99", "min": "5", "max": "600", "missing": "0"}, "updatedLabel": "AppointmentDuration"}, {"transformationsData": [{"feature_label": "AppointmentType", "transformation_label": "String Indexer"}], "feature": "AppointmentType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "*ESTABLISHED PATIENT", "max": "VIRTUAL", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AppointmentType"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EligibilityFlag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "EligibilityFlag"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ConfirmationFlag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "0.45", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "ConfirmationFlag"}, {"transformationsData": [{"feature_label": "PracticeName", "transformation_label": "String Indexer"}], "feature": "PracticeName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "MS PG1", "max": "MS PG1", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeName"}, {"transformationsData": [{"feature_label": "PracticeSpeciality", "transformation_label": "String Indexer"}], "feature": "PracticeSpeciality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "Multi-Specialty", "max": "Multi-Specialty", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeSpeciality"}, {"transformationsData": [{"feature_label": "PracticeRegion", "transformation_label": "String Indexer"}], "feature": "PracticeRegion", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "SouthEast", "max": "SouthEast", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeRegion"}, {"transformationsData": [{"feature_label": "FacilityCategory", "transformation_label": "String Indexer"}], "feature": "FacilityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "SocCLEXX", "max": "SocWesXX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCategory"}, {"transformationsData": [{"feature_label": "FacilityCity", "transformation_label": "String Indexer"}], "feature": "FacilityCity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "HOUSTON", "max": "Sugar Land", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCity"}, {"transformationsData": [{"feature_label": "FacilityState", "transformation_label": "String Indexer"}], "feature": "FacilityState", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "TX", "max": "TX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityState"}, {"transformationsData": [{"feature_label": "FacilityZip", "transformation_label": "String Indexer"}], "feature": "FacilityZip", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "771802086.76", "stddev": "1822069.11", "min": "770025351", "max": "775053948", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityZip"}, {"transformationsData": [{"feature_label": "ProviderName", "transformation_label": "String Indexer"}], "feature": "ProviderName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "NoelAhmyyy, HassaXXX", "max": "NoelYouyyy, MeagaXXX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderName"}, {"transformationsData": [{"feature_label": "Speciality", "transformation_label": "String Indexer"}], "feature": "Speciality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "Allergy/Immun", "max": "Speech Pathologists", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Speciality"}, {"transformationsData": [{"feature_label": "ProviderTaxonomyName", "transformation_label": "String Indexer"}], "feature": "ProviderTaxonomyName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "Allopathic & Osteopathic Physicians : Allergy & Immunology", "max": "Unspecified", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderTaxonomyName"}, {"transformationsData": [{"feature_label": "PayerCategory", "transformation_label": "String Indexer"}], "feature": "PayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "29246", "mean": "", "stddev": "", "min": "Aetna", "max": "Work Comp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Label_NoKeepCancel", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "0.85", "stddev": "0.35", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Label_NoKeepCancel"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NoShowCount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "1.34", "stddev": "2.74", "min": "0", "max": "161", "missing": "0"}, "updatedLabel": "NoShowCount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "KeptCount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "42.17", "stddev": "77.44", "min": "0", "max": "691", "missing": "0"}, "updatedLabel": "KeptCount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CanceledCount<=24hrs", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "3.15", "stddev": "12.05", "min": "0", "max": "381", "missing": "0"}, "updatedLabel": "CanceledCount<=24hrs"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CanceledCount>24hrs", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "0.74", "stddev": "4.73", "min": "0", "max": "151", "missing": "0"}, "updatedLabel": "CanceledCount>24hrs"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Reschedule<=24hrs", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "2.86", "stddev": "4.23", "min": "0", "max": "46", "missing": "0"}, "updatedLabel": "Reschedule<=24hrs"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Reschedule>24hrs", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "29246", "mean": "3.04", "stddev": "3.71", "min": "0", "max": "43", "missing": "0"}, "updatedLabel": "Reschedule>24hrs"}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "29246", "mean": "0.43", "stddev": "0.5", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "AppointmentDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "29246", "mean": "15.37", "stddev": "8.7", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "AppointmentDate_dayofmont..."}, {"feature": "AppointmentDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "29246", "mean": "6.49", "stddev": "3.56", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "AppointmentDate_month"}, {"feature": "AppointmentDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "29246", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "AppointmentDate_year"}, {"feature": "AppointmentType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "29246", "mean": "3.2", "stddev": "5.03", "min": "0.0", "max": "47.0", "missing": "0"}, "updatedLabel": "AppointmentType_stringind..."}, {"feature": "PracticeName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "29246", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PracticeName_stringindexe..."}, {"feature": "PracticeSpeciality_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "29246", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PracticeSpeciality_string..."}, {"feature": "PracticeRegion_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "29246", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PracticeRegion_stringinde..."}, {"feature": "FacilityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "29246", "mean": "2.88", "stddev": "2.53", "min": "0.0", "max": "20.0", "missing": "0"}, "updatedLabel": "FacilityCategory_stringin..."}, {"feature": "FacilityCity_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "29246", "mean": "1.07", "stddev": "1.03", "min": "0.0", "max": "6.0", "missing": "0"}, "updatedLabel": "FacilityCity_stringindexe..."}, {"feature": "FacilityState_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "29246", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FacilityState_stringindex..."}, {"feature": "FacilityZip_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "29246", "mean": "3.18", "stddev": "2.93", "min": "0.0", "max": "33.0", "missing": "0"}, "updatedLabel": "FacilityZip_stringindexer..."}, {"feature": "ProviderName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "29246", "mean": "13.31", "stddev": "10.6", "min": "0.0", "max": "50.0", "missing": "0"}, "updatedLabel": "ProviderName_stringindexe..."}, {"feature": "Speciality_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "29246", "mean": "0.42", "stddev": "0.73", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "Speciality_stringindexer_..."}, {"feature": "ProviderTaxonomyName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "29246", "mean": "1.33", "stddev": "1.95", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "ProviderTaxonomyName_stri..."}, {"feature": "PayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "29246", "mean": "3.26", "stddev": "2.96", "min": "0.0", "max": "14.0", "missing": "0"}, "updatedLabel": "PayerCategory_stringindex..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run shownoshowclassgbHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=Knearestneighborsclassifier(autofe, ["Gender_stringindexer", "PatientAge", "days_AppDate_AppCreatDate", "AppointmentDuration", "AppointmentType_stringindexer", "EligibilityFlag", "ConfirmationFlag", "PracticeName_stringindexer", "PracticeSpeciality_stringindexer", "PracticeRegion_stringindexer", "FacilityCategory_stringindexer", "FacilityCity_stringindexer", "FacilityState_stringindexer", "FacilityZip_stringindexer", "ProviderName_stringindexer", "Speciality_stringindexer", "ProviderTaxonomyName_stringindexer", "PayerCategory_stringindexer", "NoShowCount", "KeptCount", "CanceledCount<=24hrs", "CanceledCount>24hrs", "Reschedule<=24hrs", "Reschedule>24hrs", "AppointmentDate_dayofmonth", "AppointmentDate_month", "AppointmentDate_year"], "Label_NoKeepCancel")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
